<a href="https://colab.research.google.com/github/cheffjiu/pytorch-tutorials/blob/main/tensorqs_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\| **Tensors** \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Tensors（张量）
=======

张量是一种特殊的数据结构，与数组和矩阵非常相似。在PyTorch中，我们使用张量对模型的输入、输出以及模型参数进行编码。    

张量与 [NumPy](https://numpy.org/) 的多维数组类似，不同的是张量可以在GPU或其他硬件加速器上运行。实际上，张量和NumPy数组常常可以共享相同的底层内存，无需复制数据。  (see
`bridge-to-np-label`). 张量还针对自动求导进行了优化(我们会在 [Autograd](autogradqs_tutorial.html)这一节学习). 如果你熟悉`ndarray`，那么使用张量API会得心应手。如果不熟悉，那就继续学习吧！


In [ ]:
import torch
import numpy as np

Initializing a Tensor（初始化张量）
=====================

张量可以通过多种方式初始化。请看以下示例：

**直接从数据创建**

张量可以直接从数据创建。数据类型会自动推断。


In [ ]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

**从NumPy数组创建**

张量可以从NumPy数组创建 (反之亦然 - 见
`bridge-to-np-label`).


In [ ]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

**从另一个张量创建:**

除非显式覆盖，否则新张量将保留源张量的属性（形状、数据类型）。

In [ ]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

**使用随机值或常量值创建:**

`shape`是张量维度的元组。在以下函数中，它决定了输出张量的维度。.


In [ ]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

------------------------------------------------------------------------


Attributes of a Tensor（张量的属性）
======================

张量属性描述了它们的形状、数据类型以及存储它们的设备。

In [ ]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

------------------------------------------------------------------------


Operations on Tensors（张量的操作）
=====================

1200 多种张量运算，包括算术运算、线性代数、矩阵操作（转置、索引、切片）、采样等，都有全面的描述
[here](https://pytorch.org/docs/stable/torch.html).

这些操作中的每一项都可以在 CPU 以及诸如 CUDA、MPS、MTIA 或 XPU 等[加速器](https://pytorch.org/docs/stable/torch.html#accelerators)上运行。如果你使用的是 Colab，可以通过依次点击“运行时”>“更改运行时类型”>“GPU”来分配一个加速器。.

默认情况下，张量在CPU上创建。我们需要在检查加速器可用性之后，使用`.to`方法显式地将张量移动到加速器上。请记住，跨设备复制大型张量在时间和内存方面开销可能很大！

In [ ]:
# We move our tensor to the current accelerator if available
if torch.accelerator.is_available():
    tensor = tensor.to(torch.accelerator.current_accelerator())

尝试以下的一些操作。如果你熟悉NumPy API，那么使用Tensor API对你来说将轻而易举。

**标准的类似NumPy的索引和切片操作:**


In [ ]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

**连接张量** 你可以使用 `torch.cat` 沿着给定维度连接一系列张量。另请参阅 [torch.stack](https://pytorch.org/docs/stable/generated/torch.stack.html)，这是另一个与 `torch.cat` 略有不同的张量连接操作符。


In [ ]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

**代数运算**


In [ ]:
# This computes the matrix multiplication（矩阵乘法） between two tensors. y1, y2, y3 will have the same value
# ``tensor.T`` returns the transpose（转置） of a tensor
y1 = tensor @ tensor.T #@和matmul都是做矩阵乘法
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)


# This computes the element-wise product（对位元素相乘）. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

**单元素张量** 如果你有一个单元素张量，例如通过将张量的所有值聚合为一个值，你可以使用 `item()` 将其转换为Python数值：

In [ ]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

**原地操作**：将结果存储到操作数中的操作称为原地操作。它们以 `_` 后缀表示。例如：`x.copy_(y)`、`x.t_()` 会改变 `x`。


In [ ]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>原地操作节省了一些内存，但在计算导数时可能会出现问题，因为会立即丢失历史信息。因此，不建议使用它们。</p>

</div>



------------------------------------------------------------------------


Bridge with NumPy {bridge-to-np-label}（与Numpy的联系）
=================

CPU 上的张量和 NumPy 数组可以共享它们的底层内存位置，改变其中一个会导致另一个也发生改变。


张量转换为NumPy数组
=====================


In [ ]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

张量的变化会反映在NumPy数组中。


In [ ]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

NumPy数组转张量
=====================


In [ ]:
n = np.ones(5)
t = torch.from_numpy(n)

NumPy数组中的变化会反映在张量中。

---




In [ ]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

补充：张量形状与维度的操作
=====================

torch.view
=====================